[Dokumentacja mysql.connector](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)

Funkcja connect() tworzy połączenie z MySQL server i zwraca obiekt MySQLConnection:
```python
cnx = mysql.connector.connect(**config_dict)
```

Metoda połączenia cursor() zwraca obiekt MySQLCursor, który umożliwia manipulowanie bazą:
```python
cursor = cnx.cursor()

cursor.execute('sql_statement'[, data])
cursor.executemany('sql_statement', seq_of_data)
# w wyniku zapytania cursor może zostać użyty jako iterator lub można pobrać wyszukane wiersze
row_list = cursor.fetchall()

iterator = cursor.execute('many_sql_statement', multi=True)
```

Po dodaniu (INSERT INTO), zaktualizowaniu (UPDATE) lub usunięciu (DELETE FROM) rekordów, zmiany należy jeszcze skomitować:
```
cnx.commit()
```

Połączenie należy zakończyć:
```
cursor.close()
cnx.close()
```

In [1]:
import mysql.connector
import random

### Nawiązanie połączenia z bazą danych

Utworzenie bazy danych:
```mysql
CREATE DATABASE database_name;
```

In [2]:
db_name = 'cardb'

In [3]:
# słownik złożony z argumentów kluczowych funkcji connect()
config = {
  'user': 'root',
  'password': 'admin',
  'host': '127.0.0.1',
#   'database': db_name,
}

In [4]:
# połączenie się z serwerem (lub konkretną bazą na tym serwerze, jeżeli przekazany jest też argument database)
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [5]:
# połączenie się z konkretną bazą
# jeżeli baza nie istnieje, zostaje utworzona
try:
    cnx.database = db_name
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
    print('Something is wrong with your user name or password')
  elif err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
#     print('Database {} does not exist'.format(db_name))
    cursor.execute('CREATE DATABASE {}'.format(db_name))
    cnx.database = db_name
  else:
    print(err)

### Utworzenie tabel

Utworzenie nowej tabeli:
```mysql
CREATE TABLE table_name (
    column1 datatype constraint,
    column2 datatype constraint,
    column3 datatype constraint,
    ...
    PRIMARY KEY (column_name)
    /*CONSTRAINT contraint_name PRIMARY KEY (column_name(s))*/
    FOREIGN KEY (column_name) REFERENCES table2_name(column_name) 
    /*CONSTRAINT contraint_name FOREIGN KEY (column_name(s)) REFERENCES table2_name(column_name)*/
);
```

Utworzenie nowej tabeli na podstawie już istniejących:
```mysql
CREATE TABLE table_name AS
zapytanie;
```

In [6]:
table_dict = {}

In [7]:
table_dict['Car'] = '''
    CREATE TABLE `Car` (
    `Id` INT NOT NULL,
    `CarModel` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`Id`))
    '''

table_dict['Part'] = '''
    CREATE TABLE `Part` (
    `Id` INT NOT NULL,
    `IsFunctional` TINYINT(1) NOT NULL,
    `CarId` INT,
    PRIMARY KEY (`Id`),
    FOREIGN KEY (`CarId`) REFERENCES `Car`(`Id`))
    '''

table_dict['Wheel'] = '''
    CREATE TABLE `Wheel` (
    `PartId` INT NOT NULL,
    `WheelModel` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

table_dict['Engine'] = '''
    CREATE TABLE `Engine` (
    `PartId` INT NOT NULL,
    `EngineModel` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

In [8]:
# utworzenie tabel
# jeżeli już istnieją, pojawi się komunikat
for table_name, create_table_stmt in table_dict.items():
    try:
        cursor.execute(create_table_stmt)
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
        print('Table {} already exists'.format(table_name))
      else:
        print(err)

### Dodanie wierszy

Wstawienie nowych rekordów do tabeli:
```mysql
INSERT INTO table_name (column1, column2, column3, ...)
VALUES (value1, value2, value3, ...), (value1, value2, value3, ...), ...;
```

In [9]:
add_car_stmt = '''
    INSERT INTO Car (Id, CarModel)
    VALUES (%s, %s)
    '''

add_part_stmt = '''
    INSERT INTO Part (Id, IsFunctional, CarId)
    VALUES (%s, %s, %s)
    '''

add_wheel_stmt = '''
    INSERT INTO Wheel (PartId, WheelModel)
    VALUES (%s, %s)
    '''

add_engine_stmt = '''
    INSERT INTO Engine (PartId, EngineModel)
    VALUES (%s, %s)
    '''

In [10]:
cursor.execute('SELECT c.Id FROM Car AS c')
try:
    last_car_id = cursor.fetchall()[-1][0]
except IndexError:
    last_car_id = 0

car_total = 20
car_id_list = list(range(last_car_id + 1, last_car_id + 1 + car_total))
car_model_list = ['Renault', 'Audi', 'BMW', 'Mazda', 'Fiat']
car_list = [(car_id, random.choice(car_model_list)) for car_id in car_id_list]

cursor.execute('SELECT p.Id FROM Part AS p')
try:
    last_part_id = cursor.fetchall()[-1][0]
except IndexError:
    last_part_id = 0

part_total = 100
part_id_list = list(range(last_part_id + 1, last_part_id + 1 + part_total))
part_list = [(part_id, random.randint(0, 1), random.choice(car_id_list)) for part_id in part_id_list]

wheel_total = int(part_total * 0.7)
wheel_model_list = ['W1', 'W2', 'W3', 'W4', 'W5']
wheel_part_id_list = random.sample(part_id_list, wheel_total)
wheel_list = [(part_id, random.choice(wheel_model_list)) for part_id in wheel_part_id_list]

engine_total = part_total - wheel_total
engine_model_list = ['A157', 'B458', 'A86', 'A123', 'C18']
engine_part_id_list = list(set(part_id_list) - set(wheel_part_id_list))
engine_list = [(part_id, random.choice(engine_model_list)) for part_id in engine_part_id_list]

In [11]:
cursor.executemany(add_car_stmt, car_list)

In [12]:
cursor.executemany(add_part_stmt, part_list)

In [13]:
cursor.executemany(add_wheel_stmt, wheel_list)

In [14]:
cursor.executemany(add_engine_stmt, engine_list)

In [15]:
cnx.commit()

### Zaktualizowanie wierszy
Zmodyfikowanie istniejących rekordów w tabeli:
```mysql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

In [16]:
set_free_part_stmt = '''
    UPDATE Part AS p
    SET p.CarId = NULL
    WHERE p.Id IN {}
    '''

set_broken_part_stmt = '''
    UPDATE Part AS p
    SET p.IsFunctional = 0
    WHERE p.Id = %s
    '''

set_fixed_part_stmt = '''
    UPDATE Part AS p
    SET p.IsFunctional = 1
    WHERE p.Id = %s
    '''

In [17]:
free_part_total = 15
free_part_id_list = random.sample(part_id_list, free_part_total)
set_free_part_full_stmt = set_free_part_stmt.format(str(('%s',) * free_part_total))

In [18]:
cursor.execute(set_free_part_full_stmt, free_part_id_list)

In [19]:
fixed_part_id_list = [(part_id,) for part_id in random.sample(part_id_list, 10)]

In [20]:
cursor.executemany(set_fixed_part_stmt, fixed_part_id_list)

In [21]:
cnx.commit()

### Utworzenie widoku

Widok jest wirtualną tabelą opartą na wyniku zapytania. Można z niego korzystać tak, jak z rzeczywistych tabel:
```mysql
CREATE VIEW view_name AS
zapytanie;
```

In [22]:
# stwórz widok sprawnych samochodów
# samochód jest sprawny, jeżeli wszystkie jego części są sprawne
create_view_functional_car_stmt = '''
    CREATE VIEW FunctionalCar AS
    SELECT c.Id, c.CarModel FROM Car AS c
    WHERE c.Id IN (SELECT p.CarId FROM Part AS p
                   GROUP BY p.CarId
                   HAVING COUNT(p.IsFunctional) = SUM(p.IsFunctional))
    '''
# inny sposób:
#     SELECT c.Id, c.CarModel
#     FROM Car AS c INNER JOIN Part AS p ON c.Id = p.CarId
#     GROUP BY c.Id
#     HAVING COUNT(p.IsFunctional) = SUM(p.IsFunctional)

In [23]:
try:
    cursor.execute(create_view_functional_car_stmt)
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
    print('View already exists')
  else:
    print(err)

### Zapytania

Schemat zapytania:
```mysql
SELECT column_name(s)
FROM table_name
WHERE condition 
GROUP BY column_name(s)
HAVING condition
ORDER BY column_name(s)
LIMIT no_of_rows;
```

Połączenie tabel względem pasujących do siebie wierszy:
```mysql
SELECT column_name(s)
FROM table1
INNER JOIN table2 /*INNER/FULL OUTER/LEFT/RIGHT JOIN*/
ON table1.column_name = table2.column_name;
```

Połączenie dwóch wyników zapytań - muszą być te same kolumny:
```mysql
SELECT column_name(s) FROM table1
UNION /*UNION ALL pozwala na duplikaty*/
SELECT column_name(s) FROM table2;
```

In [24]:
def print_res_of_query(cursor):
    frame_str = '{:<12} ' * len(cursor.column_names)
    print(frame_str.format(*cursor.column_names))
    for row in cursor:
        print(frame_str.format(*row))    

* #### Znajdź sprawne samochody
Samochód jest sprawny, jeżeli wszystkie jego części są sprawne

In [25]:
get_functional_car_query = '''
    SELECT fc.Id AS CarId, fc.CarModel FROM FunctionalCar AS fc
    '''

In [26]:
cursor.execute(get_functional_car_query)
print_res_of_query(cursor)

CarId        CarModel     
5            Mazda        
14           Renault      
17           Mazda        
19           Audi         


* #### Znajdź liczbę sprawnych i niesprawnych samochodów danej marki

In [27]:
get_funct_and_nonfunct_model_car_query = '''
    SELECT c.CarModel, COUNT(fc.Id) AS FunctCar, COUNT(c.Id) - COUNT(fc.Id) AS NonfunctCar
    FROM Car AS c 
    LEFT JOIN FunctionalCar AS fc 
    ON fc.Id = c.Id
    GROUP BY c.CarModel
    '''

In [28]:
cursor.execute(get_funct_and_nonfunct_model_car_query)
print_res_of_query(cursor)

CarModel     FunctCar     NonfunctCar  
Audi         1            3            
BMW          0            3            
Fiat         0            4            
Mazda        2            5            
Renault      1            1            


* #### Znajdź liczbę sprawnych i niesprawnych części dla konkretnych samochodów
Zapisz je na dwa sposoby - w dwóch oddzielnych kolumnach i wierszach

In [29]:
# oddzielne kolumny
get_car_funct_and_nonfunct_part_query = '''
    SELECT c.Id AS CarId, c.CarModel, SUM(p.IsFunctional) AS FunctPart, COUNT(p.IsFunctional) - SUM(p.IsFunctional) AS NonfunctPart
    FROM Car AS c
    INNER JOIN Part AS p
    ON c.Id = p.CarId
    GROUP BY c.Id
    LIMIT 10
    '''

In [30]:
cursor.execute(get_car_funct_and_nonfunct_part_query)
print_res_of_query(cursor)

CarId        CarModel     FunctPart    NonfunctPart 
1            Fiat         2            2            
2            Audi         1            1            
3            Audi         3            2            
4            Mazda        1            3            
5            Mazda        3            0            
6            Mazda        4            5            
7            Fiat         1            2            
8            Mazda        1            2            
9            Mazda        3            4            
10           Fiat         0            3            


In [31]:
# oddzielne wiersze
get_car_funct_and_nonfunct_part_query = '''
    SELECT c.Id AS CarId, c.CarModel, p.IsFunctional, COUNT(p.IsFunctional) AS NoOfPart
    FROM Car AS c
    INNER JOIN Part AS p
    ON c.Id = p.CarId
    GROUP BY c.Id, p.IsFunctional
    LIMIT 10
    '''

In [32]:
cursor.execute(get_car_funct_and_nonfunct_part_query)
print_res_of_query(cursor)

CarId        CarModel     IsFunctional NoOfPart     
1            Fiat         0            2            
1            Fiat         1            2            
2            Audi         0            1            
2            Audi         1            1            
3            Audi         0            2            
3            Audi         1            3            
4            Mazda        0            3            
4            Mazda        1            1            
5            Mazda        1            3            
6            Mazda        0            5            


* #### Znajdź wszystkie luźne części
Luźne części to te, które nie są związane z żadnym samochodem

In [33]:
get_free_part_query = '''
    SELECT p.Id AS PartId, 'Wheel' AS PartType, w.WheelModel AS PartModel, p.IsFunctional
    FROM Part AS p
    INNER JOIN Wheel AS w
    ON p.Id = w.PartId
    WHERE p.CarId IS NULL
    
    UNION
    
    SELECT p.Id AS PartId, 'Engine' AS PartType, e.EngineModel AS PartModel, p.IsFunctional
    FROM Part AS p
    INNER JOIN Engine AS e
    ON p.Id = e.PartId
    WHERE p.CarId IS NULL
    ORDER BY PartId
    LIMIT 10
    '''

In [34]:
cursor.execute(get_free_part_query)
print_res_of_query(cursor)

PartId       PartType     PartModel    IsFunctional 
10           Wheel        W2           1            
15           Wheel        W4           0            
16           Engine       C18          1            
18           Wheel        W1           0            
30           Wheel        W1           1            
39           Wheel        W4           0            
40           Engine       A157         1            
51           Engine       A86          1            
52           Wheel        W1           1            
63           Wheel        W5           1            


* #### Znajdź samochody bez części

In [35]:
get_car_no_part_query = '''
    SELECT c.Id, c.CarModel FROM Car AS c
    WHERE c.Id NOT IN (SELECT DISTINCT p.CarId FROM Part AS p 
                       WHERE p.CarId IS NOT NULL)
    LIMIT 10
    '''

In [36]:
cursor.execute(get_car_no_part_query)
print_res_of_query(cursor)

Id           CarModel     


* #### Znajdź wszystkie części związane z samochodami

In [37]:
get_car_part_query = '''
    SELECT FullPart.CarId, c.CarModel, FullPart.PartType, FullPart.PartModel, FullPart.IsFunctional
    FROM Car AS c
    INNER JOIN (
        SELECT w.WheelModel AS PartModel, 'Wheel' AS PartType, p.IsFunctional, p.CarId
        FROM Part AS p INNER JOIN Wheel AS w ON p.Id = w.PartId
        UNION
        SELECT e.EngineModel AS PartModel, 'Engine' AS PartType, p.IsFunctional, p.CarId
        FROM Part AS p INNER JOIN Engine AS e ON p.Id = e.PartId) 
        AS FullPart 
    ON FullPart.CarId = c.Id
    ORDER BY FullPart.CarId
    LIMIT 20
    '''

In [38]:
cursor.execute(get_car_part_query)
print_res_of_query(cursor)

CarId        CarModel     PartType     PartModel    IsFunctional 
1            Fiat         Engine       C18          1            
1            Fiat         Wheel        W2           0            
1            Fiat         Wheel        W5           1            
1            Fiat         Engine       B458         0            
2            Audi         Wheel        W2           0            
2            Audi         Wheel        W1           1            
3            Audi         Wheel        W5           0            
3            Audi         Engine       B458         0            
3            Audi         Wheel        W2           1            
3            Audi         Wheel        W3           1            
3            Audi         Engine       A157         1            
4            Mazda        Engine       A86          0            
4            Mazda        Engine       A123         0            
4            Mazda        Engine       A86          1            
4         

* #### Znajdź dostępne modele i ich liczbę

In [39]:
get_model_query = '''
    SELECT {0}Model, COUNT(*) AS Count FROM {0}
    GROUP BY {0}Model
    '''

table_name_list = ['Car', 'Wheel', 'Engine']

In [40]:
result_iter = cursor.execute('; '.join(map(get_model_query.format, table_name_list)), multi=True)
frame_str = '{:%s}: {}' % (max(map(len, table_name_list)) + 1)
for table_name, result in zip(table_name_list, result_iter):
    print(frame_str.format(table_name, result.fetchall()))

Car    : [('Audi', 4), ('BMW', 3), ('Fiat', 4), ('Mazda', 7), ('Renault', 2)]
Wheel  : [('W1', 11), ('W2', 15), ('W3', 14), ('W4', 17), ('W5', 13)]
Engine : [('A123', 4), ('A157', 4), ('A86', 7), ('B458', 6), ('C18', 9)]


### Usunięcie wierszy

```mysql
DELETE FROM table_name
WHERE condition;
```

### Usunięcie tabeli

```mysql
DROP TABLE table_name;
```

### Usunięcie bazy danych

```mysql
DROP DATABASE database_name;
```

In [41]:
# try:
#     cursor.execute('DROP DATABASE {}'.format(db_name))
# except mysql.connector.Error as err:
#     print(err)

### Zakończenie połączenia

In [42]:
cursor.close()
cnx.close()